In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [7]:
df_1 = pd.read_csv("train1.csv")
df_2 = pd.read_csv("train2.csv")
df_3 = pd.read_csv("train3.csv")

df = pd.concat([df_1, df_2, df_3],axis=1)

df

,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,HR,pNN25,pNN50,SD1,...,condition,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,885.157845,853.763730,140.972741,15.554505,15.553371,9.063146,69.499952,11.133333,0.533333,11.001565,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,939.425371,948.357865,81.317742,12.964439,12.964195,6.272369,64.363150,5.600000,0.000000,9.170129,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,898.186047,907.006860,84.497236,16.305279,16.305274,5.182201,67.450066,13.066667,0.200000,11.533417,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,881.757864,893.460030,90.370537,15.720468,15.720068,5.748591,68.809562,11.800000,0.133333,11.119476,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,809.625331,811.184865,62.766242,19.213819,19.213657,3.266724,74.565728,20.200000,0.200000,13.590641,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369284,721.396910,721.533965,36.377559,9.609941,9.609936,3.785409,83.384647,0.933333,0.000000,6.797519,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
369285,984.266492,978.622945,74.918433,13.762274,13.761705,5.443754,61.314243,6.333333,0.200000,9.734243,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
369286,1025.499743,1024.968400,95.309200,23.051254,23.050395,4.134664,59.028594,30.400000,2.066667,16.304530,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
369287,798.123167,803.559610,78.449897,16.736686,16.736657,4.687302,75.978628,10.000000,1.466667,11.838553,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
